<a href="https://colab.research.google.com/github/HaribanshMahato/Aiml/blob/main/Aiml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.6 MB/s eta 0:00:00


In [6]:
from google.colab import files
uploaded = files.upload()


Saving app.py to app (1).py
Saving label_encoder.pkl to label_encoder.pkl
Saving model.pkl to model.pkl
Saving symptom_vocab.pkl to symptom_vocab.pkl


In [ ]:
import pandas as pd
from flask import Flask, render_template, request
import joblib

app = Flask(__name__)

# Load the trained model, label encoder, and symptom vocabulary
model = joblib.load('./models/model.pkl')
label_encoder = joblib.load('./models/label_encoder.pkl')
symptom_list = joblib.load('./models/symptom_vocab.pkl')  # Assuming you saved symptom list here

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    if request.method == 'POST':
        # Get selected symptoms from the form
        symptoms = [request.form.get(f'symptom{i}') for i in range(1, 18)]
        # Clean: lowercase, strip, and remove empty
        symptoms = [s.lower().strip() for s in symptoms if s and s.strip() != '']

        # Create a binary input vector matching symptom_list order and length
        input_vector = [0] * len(symptom_list)
        for symptom in symptoms:
            if symptom in symptom_list:
                idx = symptom_list.index(symptom)
                input_vector[idx] = 1
            else:
                # You may want to handle unknown symptoms here
                pass

        # Predict encoded label
        pred_label = model.predict([input_vector])[0]

        # Decode label to disease name
        prediction = label_encoder.inverse_transform([pred_label])[0]

    # Render the template with symptom_list and prediction if any
    return render_template('index.html', symptom_list=symptom_list, prediction=prediction)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)

# Load model files (assumed to be in the same directory)
model = joblib.load('model.pkl')
label_encoder = joblib.load('label_encoder.pkl')
symptom_list = joblib.load('symptom_vocab.pkl')

@app.route("/")
def index():
    return "Welcome to the Disease Prediction API!"

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()
        symptoms = data.get("symptoms")

        # Convert symptoms to binary input vector
        input_vector = np.array([1 if s in symptoms else 0 for s in symptom_list]).reshape(1, -1)

        prediction = model.predict(input_vector)
        predicted_disease = label_encoder.inverse_transform(prediction)[0]

        return jsonify({"prediction": predicted_disease})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run()


In [2]:
!pip install flask-ngrok

! python app.py

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/content/app.py", line 1, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.11/dist-packages/pandas/__init__.py", line 37, in <module>
  File "/usr/local/lib/python3.11/dist-packages/pandas/_config/__init__.py", line 20, in <module>
^C
